<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/cancer_validation_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_validation_ensemble

> Purpose of this notebook is simply to run exploratory experiments to compare ensembling: 2 pretrained models under 3 types of initial weights: (BT,BT),(Sup,Sup),(BT,Sup)

In [ ]:
#| default_exp cancer_validation_ensemble

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [ ]:
#| hide

import os
from google.colab import drive

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():

    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')

    os.chdir('cancer-proj')
    
    os.system('pip install .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

## Load the data

In [ ]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [ ]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [ ]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [ ]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [ ]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

Need to run a few exploratory experiments. Based on the results, next is to run some systematic experiments, probably with W and B... Or final results...

## Experiment: Check how initial_weights=BT_pretrain goes, when using new default fine_tuning:

In [ ]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    #self.unfreeze() #don't unfreeze the resnet. We are fitting training the encoder head + projector
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end

In [41]:
#| export

class NLM(nn.Module):
    "Basic nonlinear "
    def __init__(self,encoder,device='cuda'):
        super().__init__()

        self.encoder=encoder
        # self.fc1 = nn.Linear(2048,2048)
        # self.bn1 = nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        # self.relu = nn.ReLU(inplace=True)
        # self.head=nn.Linear(2048,9)

        self.head = sequential(nn.Linear(2048,2048),nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                               nn.ReLU(inplace=True),nn.Linear(2048,9))
        
        self.device = torch.device(device)
        self.to(self.device)


    def forward(self,x):
        x=self.encoder(x)
        x=self.head(x)

        # x=self.encoder(x)
        # x=self.fc1(x)
        # x=self.bn1(x)
        # x=self.relu(x)
        # x=self.head(x)
        return x

# First we verify that adding non linear head still works. This should only take 1 run: 

In [40]:
my_splitter??

## Patch in new definition of fine_tune: We are just replacing LM with NLM. Also patch in new `main_train.fit`: just adding freeze epochs. Probably need to edit base class and add freeze_epochs capacity

In [54]:
@patch
def fine_tune(self:main_train):


    #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

    try: 
        encoder = self.encoder
    
    except AttributeError:
        _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)

    print('adding NLM head!')
    model = NLM(self.encoder)
    learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

    #debugging
    #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

    main_train.fit(learn,fit_type='fine_tune',epochs=self.numfit,freeze_epochs=3,initial_weights=self.initial_weights) #fine tuning (don't confuse this with fit policy!)
    scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)
    #metrics dict will have f1 score, auc etc etc
    metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
    auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
    metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval

    #torch.save(model.state_dict(), self.tuned_model_path)
    return metrics #


@patch_to(main_train)
def fit(learn,fit_type,epochs,freeze_epochs,initial_weights):
    """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
    to fine_tune:bt_pretrain"""

    print(f'patched ok, freeze_epochs={freeze_epochs}')

    if fit_type == 'encoder_fine_tune': #i.e. barlow twins

        learn.encoder_fine_tune(epochs,freeze_epochs=freeze_epochs) 

    elif fit_type == 'fine_tune':
        
        #elif initial_weights == 'supervised_pretrain':
        learn.linear_fine_tune(epochs,freeze_epochs=freeze_epochs) 


## Test: make sure that it freezes the appropriate part of model:

In [43]:
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = NLM(encoder)
test_eq(len(my_splitter(model)),2)
test_eq(len(my_splitter_bt(bt_model)),2)

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('body should be frozen, (sans batchnorm) head unfrozen')
learn.summary()

Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


body should be frozen, (sans batchnorm) head unfrozen


NLM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                              

## Check that adding head works ok

In [55]:

initial_weights = 'supervised_pretrain'
pretrain=False
numfit=50
num_epochs=10
#main.freeze_encoder=False

main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
        aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
        initial_weights=initial_weights,pretrain=pretrain,num_epochs=num_epochs,numfit=numfit,
        print_report=True,
                 )


metrics = main()



adding NLM head!
patched ok, freeze_epochs=3


epoch,train_loss,valid_loss,time
0,2.266208,None,00:06
1,2.056458,None,00:06
2,1.805774,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,1.059653,None,00:07
1,0.977633,None,00:07
2,0.919163,None,00:06
3,0.872374,None,00:06
4,0.819349,None,00:06
5,0.755209,None,00:06
6,0.698958,None,00:07
7,0.642834,None,00:06
8,0.591585,None,00:07
9,0.546778,None,00:06


                            precision    recall  f1-score   support

         actinic keratosis       0.59      0.65      0.62        20
      basal cell carcinoma       0.88      0.75      0.81        20
            dermatofibroma       0.89      0.89      0.89        19
                  melanoma       0.56      0.45      0.50        20
                     nevus       0.69      0.90      0.78        20
pigmented benign keratosis       0.79      0.55      0.65        20
      seborrheic keratosis       0.47      0.47      0.47        15
   squamous cell carcinoma       0.52      0.65      0.58        20
           vascular lesion       0.90      0.90      0.90        20

                  accuracy                           0.70       174
                 macro avg       0.70      0.69      0.69       174
              weighted avg       0.71      0.70      0.69       174



## Based on experimentation below seems we might need to train BT head for longer?

In [ ]:

initial_weights = 'bt_pretrain'
pretrain=False
numfit=50
num_epochs=10
#main.freeze_encoder=False

main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
        aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
        initial_weights=initial_weights,pretrain=pretrain,num_epochs=num_epochs,numfit=numfit,
        print_report=True,
                 )


metrics = main()

In [56]:
metrics['acc']

0.6954023241996765

## Note: adding simple non-linear head got 0.6954 (supervised). Also we set freeze_numfit=3. So this is our baseline. 

## Ensemble experiment: Let's train 3 models, for each kind of initial weights. Then we can compare, pairwise: ensembling 'within' models (i.e. same kind of initial weights) vs. 'between' models (i.e. different kinds of initial weights).

In [60]:
#Get supervised_pretrain scores:

initial_weights = 'supervised_pretrain'
pretrain=False
numfit=50
num_epochs='na'

supervised_metrics = {0:metrics} #can reuse what we already have 

for i in range(2):

    main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
            aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
            initial_weights=initial_weights,pretrain=pretrain,num_epochs=num_epochs,numfit=numfit,
            print_report=True,
                    )
    
    supervised_metrics[i] = main()


#Get bt_pretrain scores:
initial_weights = 'bt_pretrain'
bt_metrics = {}

for j in range(3): #one more than above, as we already have earlier result

    test_eq(initial_weights,'bt_pretrain')
    test_eq(numfit,50)

    main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
        aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
        initial_weights=initial_weights,pretrain=pretrain,num_epochs=num_epochs,numfit=numfit,
        print_report=True,
                )
    
    bt_metrics[j] = main()


#Save results

results = {'bt_metrics':bt_metrics,'supervised_metrics':supervised_metrics,'docstring':'freeze epochs was 3, we used a simple nonlinear head, numfit was 50, pretrain was false'}

save_dict_to_gdrive(results,'ensemble_base_metrics')

adding NLM head!
patched ok, freeze_epochs=3


epoch,train_loss,valid_loss,time
0,2.215147,None,00:06
1,2.059524,None,00:06
2,1.850688,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,1.167445,None,00:07
1,1.096232,None,00:06
2,0.994915,None,00:07
3,0.913267,None,00:06
4,0.843889,None,00:07
5,0.780045,None,00:06
6,0.717308,None,00:06
7,0.659265,None,00:07
8,0.607973,None,00:06
9,0.561728,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.48      0.55      0.51        20
      basal cell carcinoma       0.60      0.60      0.60        20
            dermatofibroma       0.84      0.84      0.84        19
                  melanoma       0.47      0.40      0.43        20
                     nevus       0.71      0.75      0.73        20
pigmented benign keratosis       0.61      0.55      0.58        20
      seborrheic keratosis       0.47      0.60      0.53        15
   squamous cell carcinoma       0.59      0.50      0.54        20
           vascular lesion       0.85      0.85      0.85        20

                  accuracy                           0.63       174
                 macro avg       0.63      0.63      0.62       174
              weighted avg       0.63      0.63      0.63       174

adding NLM head!
patched ok, freeze_epochs=3


...

adding NLM head!
patched ok, freeze_epochs=3


epoch,train_loss,valid_loss,time
0,2.189130,None,00:06
1,1.974070,None,00:06
2,1.705241,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,0.941889,None,00:07
1,0.851187,None,00:07
2,0.809869,None,00:07
3,0.781297,None,00:07
4,0.732379,None,00:06
5,0.695629,None,00:07
6,0.654405,None,00:06
7,0.618348,None,00:06
8,0.585315,None,00:07
9,0.544475,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.55      0.60      0.57        20
      basal cell carcinoma       0.64      0.45      0.53        20
            dermatofibroma       0.68      0.79      0.73        19
                  melanoma       0.50      0.40      0.44        20
                     nevus       0.62      0.65      0.63        20
pigmented benign keratosis       0.50      0.40      0.44        20
      seborrheic keratosis       0.47      0.47      0.47        15
   squamous cell carcinoma       0.43      0.60      0.50        20
           vascular lesion       0.80      0.80      0.80        20

                  accuracy                           0.57       174
                 macro avg       0.58      0.57      0.57       174
              weighted avg       0.58      0.57      0.57       174



#Evaluate ensemble results: 

In [65]:
list(supervised_metrics.keys())

[0, 1]

In [72]:
for i in range(3):
    print(f"supervised_metrics acc is:{supervised_metrics[i]['acc']}")

for i in range(3):

    print(f"bt_metrics acc is:{bt_metrics[i]['acc']}")

supervised_metrics acc is:0.6954023241996765
supervised_metrics acc is:0.6264367699623108
supervised_metrics acc is:0.6666666865348816
bt_metrics acc is:0.6091954112052917
bt_metrics acc is:0.6264367699623108
bt_metrics acc is:0.5747126340866089


In [ ]:
#| hide

# #old supervised baseline (with fine tune)

# tem = {0: 0.6724137663841248,
#  1: 0.7126436829566956,
#  2: 0.6724137663841248,
#  3: 0.6321839094161987,
#  4: 0.6896551847457886}

# from statistics import mean
# mean(list(tem.values()))